In [2]:
import sys
sys.path.append("../..") 

from custom_scripts import database

In [3]:
### Table of unique carrier identifiers present in the test samples
carriers = database.query(""" 
                            SELECT op_unique_carrier, COUNT(*)
                                FROM flights_test
                                WHERE fl_date = ANY('{2020-01-01, 2020-01-02, 2020-01-03, 2020-01-04, 2020-01-05, 2020-01-06, 2020-01-07}')
                                GROUP BY op_unique_carrier;
                            """)
carriers.to_csv('../../data/preprocessing/unique_carriers.csv', index=False)
carriers.head(1)

,op_unique_carrier,count
0,9E,4942


In [4]:
### Table of flight numbers present in the test samples, and a count of their frequency
test_flight_numbers = database.query("""
                                SELECT op_carrier_fl_num, COUNT(*) 
                                    FROM flights_test
                                    WHERE fl_date = ANY('{{2020-01-31, 2020-01-30, 2020-01-29, 2020-01-28, 2020-01-27, 2020-01-26, 2020-01-25}}')
                                    GROUP BY op_carrier_fl_num;
                                """)
test_flight_numbers.to_csv('../../data/preprocessing/test_flight_numbers.csv', index=False)
test_flight_numbers.head(1)

,op_carrier_fl_num,count
0,6114,14


In [6]:
### Historic average delay times by flight number
avg_flight_delays_by_flight_number = database.query("""
                                SELECT  op_carrier_fl_num,
                                        AVG(arr_delay) AS "avg_arr_delay",
                                        AVG(carrier_delay) AS "avg_carrier_delay", 
                                        AVG(weather_delay) AS "avg_weather_delay", 
                                        AVG(nas_delay) AS "avg_nas_delay", 
                                        AVG(late_aircraft_delay) AS "avg_late_aircraft_delay",
                                        AVG(dep_delay) AS "avg_dep_delay"
                                    FROM flights
                                    GROUP BY op_carrier_fl_num;
                                """)
avg_flight_delays_by_flight_number.to_csv('../../data/preprocessing/average_flight_delays_by_flight_number.csv', index=False)
avg_flight_delays_by_flight_number.head(1)


,op_carrier_fl_num,avg_arr_delay,avg_carrier_delay,avg_weather_delay,avg_nas_delay,avg_late_aircraft_delay,avg_dep_delay
0,1,-5.716606,24.254237,1.391949,12.355932,6.618644,2.7734


In [8]:
### Average flight delays grouped by destination airport ID
avg_flight_delays_by_destination_airport = database.query("""
                                SELECT  dest_airport_id, 
                                        AVG(arr_delay) AS "avg_arr_delay",
                                        AVG(carrier_delay) AS "avg_carrier_delay", 
                                        AVG(weather_delay) AS "avg_weather_delay", 
                                        AVG(nas_delay) AS "avg_nas_delay", 
                                        AVG(late_aircraft_delay) AS "avg_late_aircraft_delay",
                                        AVG(dep_delay) AS "avg_dep_delay"
                                    FROM flights
                                    GROUP BY dest_airport_id;
                                """)
avg_flight_delays_by_destination_airport.to_csv('../../data/preprocessing/avg_flight_delays_by_destination_airport.csv', index=False)
avg_flight_delays_by_destination_airport.head(1)

,dest_airport_id,avg_arr_delay,avg_carrier_delay,avg_weather_delay,avg_nas_delay,avg_late_aircraft_delay,avg_dep_delay
0,10135,4.592501,23.426673,5.866636,10.776352,25.921632,9.510175


In [9]:
### Average flight delays grouped by origin airport ID
avg_flight_delays_by_origin_airport = database.query("""
                                SELECT  origin_airport_id, 
                                        AVG(arr_delay) AS "avg_arr_delay",
                                        AVG(carrier_delay) AS "avg_carrier_delay", 
                                        AVG(weather_delay) AS "avg_weather_delay", 
                                        AVG(nas_delay) AS "avg_nas_delay", 
                                        AVG(late_aircraft_delay) AS "avg_late_aircraft_delay",
                                        AVG(dep_delay) AS "avg_dep_delay"
                                    FROM flights
                                    GROUP BY origin_airport_id;
                                """)
avg_flight_delays_by_origin_airport.to_csv('../../data/preprocessing/avg_flight_delays_by_origin_airport.csv', index=False)
avg_flight_delays_by_origin_airport.head(1)

,origin_airport_id,avg_arr_delay,avg_carrier_delay,avg_weather_delay,avg_nas_delay,avg_late_aircraft_delay,avg_dep_delay
0,10135,7.024949,24.29115,4.138496,17.511504,33.039823,10.58383


In [3]:
### Average flight delays grouped by carrier ID
avg_flight_delays_by_carrier = database.query("""
                                SELECT  op_unique_carrier, 
                                        AVG(arr_delay) AS "avg_arr_delay",
                                        AVG(carrier_delay) AS "avg_carrier_delay", 
                                        AVG(weather_delay) AS "avg_weather_delay", 
                                        AVG(nas_delay) AS "avg_nas_delay", 
                                        AVG(late_aircraft_delay) AS "avg_late_aircraft_delay",
                                        AVG(dep_delay) AS "avg_dep_delay"
                                    FROM flights
                                    GROUP BY op_unique_carrier;
                                """)
avg_flight_delays_by_carrier.to_csv('../../data/preprocessing/avg_flight_delays_by_carrier.csv', index=False)
avg_flight_delays_by_carrier.head(1)

,op_unique_carrier,avg_arr_delay,avg_carrier_delay,avg_weather_delay,avg_nas_delay,avg_late_aircraft_delay,avg_dep_delay
0,9E,3.788254,20.54492,5.287241,21.227495,31.674297,10.550252


In [3]:
avg_flight_delays_by_day_of_year = database.query("""
                                  SELECT  extract(doy from TO_DATE(fl_date,'YYYY-MM-DD')) AS "day_of_year", 
                                        AVG(arr_delay) AS "avg_arr_delay",
                                        AVG(carrier_delay) AS "avg_carrier_delay", 
                                        AVG(weather_delay) AS "avg_weather_delay", 
                                        AVG(nas_delay) AS "avg_nas_delay", 
                                        AVG(late_aircraft_delay) AS "avg_late_aircraft_delay",
                                        AVG(dep_delay) AS "avg_dep_delay"
                                    FROM flights
                                    GROUP BY day_of_year;
                                """)
avg_flight_delays_by_day_of_year.to_csv('../../data/preprocessing/avg_flight_delays_by_day_of_year.csv', index=False)
avg_flight_delays_by_day_of_year.head(1)

,day_of_year,avg_arr_delay,avg_carrier_delay,avg_weather_delay,avg_nas_delay,avg_late_aircraft_delay,avg_dep_delay
0,1.0,10.488622,24.226509,3.270192,9.308377,28.867853,16.427019
